# Agent

\begin{align}
p(r_t,u_t,\omega_t,a_t | \omega_{t-1}) \propto \underbrace{p(r_t|u_t) p(u_t|\omega_t)}_{\text{likelihood}} \underbrace{p(\omega_t | \omega_{t-1}, a_t)}_{\text{state transition}}\underbrace{p(a_t)}_{\text{control}}
\end{align}

\begin{align}
p(r_t,u_t,\omega_t | \omega_{t-1}) \propto p(r_t|\omega_t)p(\omega_t | \omega_{t-1}, u_t)p(u_t)
\end{align}

In [1]:
using ForneyLab
using ProgressMeter
using LinearAlgebra
using Plots

In [ ]:
# RW baseline
function create_agent(horizon)
    # Building the model
    n_samples = horizon
    fg = FactorGraph()

    # State prior
    @RV ω_0 ~ GaussianMeanPrecision(placeholder(:m_ω_0), placeholder(:w_ω_0))

    @RV a ~ GaussianMeanPrecision(placeholder(:m_a), placeholder(:w_a))

    # Transition and observation model
    ω = Vector{Variable}(undef, n_samples)
    u = Vector{Variable}(undef, n_samples)
    r = Vector{Variable}(undef, n_samples)

    ω_i_min = ω_0
    for i in 1:n_samples

        @RV ω[i] = ω_i_min + a

        f(z) = 1/(1+ exp(-z))
        @RV u[i] ~ Nonlinear{Sampling}(ω[i], g=f)
        @RV r[i] ~ GaussianMeanPrecision(u[i], 100.0)

        # Data placeholder
        placeholder(r[i], :r, index=i)

        # Reset state for next step
        ω_i_min = ω[i]
    end

end


In [15]:
draw()

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.40.1 (20161225.0304)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 3915994815025300801 -->
 
 3915994815025300801 
 
 clamp_2 
 
<!-- 9542302890259382853 -->
 
 9542302890259382853 
 
 + 
 addition_1 
 
<!-- 12923314422989553919 -->
 
 12923314422989553919 
 
 𝒩 
 gaussianmeanprecision_1 
 
<!-- 9542302890259382853--12923314422989553919 -->
 
 9542302890259382853--12923314422989553919 
 
 ω_0 
 1 out 
 2 in1 
 
<!-- 15948200860911817487 -->
 
 15948200860911817487 
 
 + 
 addition_2 
 
<!-- 9398039081215869110 -->
 
 9398039081215869110 
 
 𝒩 
 gaussianmeanprecision_2 
 
<!-- 14986582400706139117 -->
 
 14986582400706139117 
 
 placeholder_w_u 
 
<!-- 9398039081215869110--14986582400706139117 -->
 
 9398039081215869110--14986582400706139117 
 
 w_u 
 1 out 
 3 w 
 
<!-- 13934258549476598602 -->
 
 13934258549476598602 
 
 placeholder_m_u 
 
<!-- 9398039081215869110--13934258549476598602 -->
 
 9398039081215869110--13934258549476598602 
 
 m_u 
 1 out 
 2 m 
 
<!-- 13827969823374406485 -->
 
 13827969823374406485 
 
 placeholder_m_ω_0 
 
<!-- 14618576517301047757 -->
 
 14618576517301047757 
 
 g{Sampling} 
 nonlinear_1 
 
<!-- 14766985876718743537 -->
 
 14766985876718743537 
 
 placeholder_r_1 
 
<!-- 5481716205405964635 -->
 
 5481716205405964635 
 
 𝒩 
 gaussianmeanprecision_3 
 
<!-- 14766985876718743537--5481716205405964635 -->
 
 14766985876718743537--5481716205405964635 
 
 r_1 
 1 out 
 1 out 
 
<!-- 1380113659052177822 -->
 
 1380113659052177822 
 
 placeholder_r_2 
 
<!-- 3509632161792417769 -->
 
 3509632161792417769 
 
 𝒩 
 gaussianmeanprecision_4 
 
<!-- 1380113659052177822--3509632161792417769 -->
 
 1380113659052177822--3509632161792417769 
 
 r_2 
 1 out 
 1 out 
 
<!-- 3509632161792417769--3915994815025300801 -->
 
 3509632161792417769--3915994815025300801 
 
 clamp_2 
 1 out 
 3 w 
 
<!-- 9576812937011747547 -->
 
 9576812937011747547 
 
 g{Sampling} 
 nonlinear_2 
 
<!-- 3509632161792417769--9576812937011747547 -->
 
 3509632161792417769--9576812937011747547 
 
 u_2 
 1 out 
 2 m 
 
<!-- 5481716205405964635--14618576517301047757 -->
 
 5481716205405964635--14618576517301047757 
 
 u_1 
 1 out 
 2 m 
 
<!-- 6680231799468340605 -->
 
 6680231799468340605 
 
 clamp_1 
 
<!-- 5481716205405964635--6680231799468340605 -->
 
 5481716205405964635--6680231799468340605 
 
 clamp_1 
 1 out 
 3 w 
 
<!-- 841237179330970095 -->
 
 841237179330970095 
 
 = 
 equ_a_1 
 
<!-- 841237179330970095--9542302890259382853 -->
 
 841237179330970095--9542302890259382853 
 
 a 
 3 in2 
 2 2 
 
<!-- 841237179330970095--15948200860911817487 -->
 
 841237179330970095--15948200860911817487 
 
 a 
 3 in2 
 3 3 
 
<!-- 841237179330970095--9398039081215869110 -->
 
 841237179330970095--9398039081215869110 
 
 a 
 1 out 
 1 1 
 
<!-- 5171352716156894021 -->
 
 5171352716156894021 
 
 placeholder_w_ω_0 
 
<!-- 9576812937011747547--15948200860911817487 -->
 
 9576812937011747547--15948200860911817487 
 
 ω_2 
 1 out 
 2 in1 
 
<!-- 13885930320881843369 -->
 
 13885930320881843369 
 
 = 
 equ_ω_1_1 
 
<!-- 13885930320881843369--9542302890259382853 -->
 
 13885930320881843369--9542302890259382853 
 
 ω_1 
 1 out 
 1 1 
 
<!-- 13885930320881843369--15948200860911817487 -->
 
 13885930320881843369--15948200860911817487 
 
 ω_1 
 2 in1 
 3 3 
 
<!-- 13885930320881843369--14618576517301047757 -->
 
 13885930320881843369--14618576517301047757 
 
 ω_1 
 2 in1 
 2 2 
 
<!-- 12923314422989553919--13827969823374406485 -->
 
 12923314422989553919--13827969823374406485 
 
 m_ω_0 
 1 out 
 2 m 
 
<!-- 12923314422989553919--5171352716156894021 -->
 
 12923314422989553919--5171352716156894021 
 
 w_ω_0 
 1 out 
 3 w